### GPU
GPU(Graphics Processing Unit) - устройство, спроектированное специально для того, чтобы обрабатывать 3d изображения.
Отличие этих вычислений от того, что мы делали раньше заключается в том, что для того, чтобы отобразить картинку, нужно сделать большое количество относительно простых операций с большим количеством объектов(например, рассчитать для каждой точки на плоскости, видна ли она в данный момент времени). Для этого используется большое количество простых ядер, работающих на меньшей частоте, чем те, что используются в процессорах.

В определенных задачах это может давать серьёзный прирост по производительности, в других же, наоборот эффективней использовать процессор.

### Задачи для GPU (параллельные, однородные вычисления):
- Матричные операции
- Обучение нейронных сетей
- Обработка изображений
- Симуляции физики (расчёт взаимодействия множества частиц)

### Задачи для CPU (последовательные, сложная логика):
- Парсинг и обработка текста
- Работа с базами данных
- Компиляция кода (сложный анализ, оптимизации)
- Сортировка и поиск в небольших данных
- Рекурсивные алгоритмы (обход деревьев, графов)


### Итог:
- GPU: много простых одинаковых операций над большими данными
- CPU: сложная логика, ветвления, последовательные зависимости


## История
Изначально подобные схемы появились как отрисовщики изображения. Они ходили по памяти компьютера, куда процессор клал данные, и отрисовывали изображение на экран. Это было сделано для того, чтобы CPU занимался последовательными расчётами, не прерываясь на относительно простые, но частые задачи отрисовки кадров.

Затем, первые GPU научились выполнять графические примитивы: отрисовка линий, заливка(благодаря специальным командам, оптимизированным для этих задач, ускоряется выполнение: там, где процессор последовательно отрисовывал каждый пиксель, GPU может сразу взять 100 и выполнить простую операцию для них). Появилась возможность выполнять сторонние вычисления(не только графические). 

Далее все шло к тому, чтобы сделать GPU как можно более гибкими: они научились выполнять циклы, операции для чисел с плавающей точкой.

Поэтому можно сделать следующий вывод: GPU -- много-много медленных CPU, к которым мы можем обращаться одновременно(продвинутая многопоточность), однако есть некоторые ограничения, связанные с специализацией данных схем.

Сейчас самой популярной платформой является Nvidia CUDA(Compute Unified Device Architecture) - платформа для параллельных вычислений, под неё оптимизированно большое количество библиотек, поэтому давайте рассмотрим именно её.

Более детельно посмотрим на устройство Nvidia A100:

<img src="a100_sm.png" width="500" alt="description">




Чип разбит на SM(Streaming Multiprocessor) - вычислительные блоки, каждый из который отвечает за выполнение потоков на выделенных ядрах.
Ядра делятся на несколько типов:
- CUDA - выполняют арифметические операции с int32, float32/64 
- Tensor Core - специализированные ядра для матричных операций
- RT-cores - ядра, для трассировки лучей(ускоренные проверки с лучами и т.п.)

Когда поступает какая-то задача, то она разбивается на блоки потоков(каждый блок идёт на отдельный SM), эти блоки разбиваются на warp-ы -- 32 потока.
Уже warp-ы исполняются. Причём задачи в одном warp -- одинаковые, т.е. мы можем сложить 32 int-a.

Warp Scheduler отслеживает то, какие warp-ы готовы к исполнению и отправляет их к Dispatch Unit.(warp-у нужно загрузить из более медленной памяти какие-то данные -- он простаивает, если же есть другой уже готовый warp, то мы переключаемся на него)
Dispatch Unit уже определяет, где и на каких ядрах будет выполняться данный warp. Так же он обращается к L0 кэшу за командой для текущего warp-a.

По мере вычисления одного warp-а мы можем обращаться к register file - наиболее быстрой памяти.


Иерархия памяти:
| Тип памяти | Описание | Время обращения, тактов |
|------------|----------|------------------------|
| DRAM | память компьютера | 2000+ |
| Global memory(видео память) | память, общая для всего GPU, её может быть порядка нескольких гигабайт | 200-400 |
| L2 | служит для обмена L1 с глобальной памятью | 100-200 |
| Shared memory(L1) | общая для блока потоков | 20-30 |
| Register File | память, выделенная под выполнение конкретного warp | 1-2 |
| L0 | хранилище команд | 1-2 |


## Numba CUDA - базовое руководство

Для работы с CUDA можно использовать разные библиотеки. Давайте посмотрим на numba:

In [1]:
from numba import cuda
import numpy as np

cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 4
                              PCI Bus ID: 0
                                    UUID: GPU-517bb309-2f01-17b3-2703-6fa5306ee38c
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

### Простое сложение векторов

In [2]:
@cuda.jit
def add_kernel(x, y, out):
    idx = cuda.grid(1)
    if idx < x.size:
        out[idx] = x[idx] + y[idx]

In [3]:
n = 1000000
x = np.ones(n)
y = np.ones(n) * 2
out = np.zeros(n)

threads_per_block = 256
blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:937: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


### Как стоит писать код внутри функции

1. Каждый поток должен работать с одним элементом данных
2. Избегайте ветвлений (if/else)
3. Используйте локальные переменные вместо глобальной памяти
4. Минимизируйте обращения к глобальной памяти
5. Используйте shared memory для кэширования данных

### Определение позиции внутри сетки потоков

Для того, чтобы понимать, с каким элементом массива будет работать данная функция, можно привязаться к сетке потоков.
Чтобы определить своё положение в сетке используем следующие функции:

In [4]:
@cuda.jit
def test_func(out):
    idx = cuda.grid(1)
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    gdim = cuda.gridDim.x
    
    if idx < out.size:
        out[idx] = tid * 1000 + bid * 100 + bdim * 10 + gdim

In [5]:
out = np.zeros(10)
test_func[2, 4](out)
print(out)

[  42. 1042. 2042. 3042.  142. 1142. 2142. 3142.    0.    0.]


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:937: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


### Многомерные массивы (2D)

In [6]:
@cuda.jit
def matrix_op(matrix):
    x, y = cuda.grid(2)
    if x < matrix.shape[0] and y < matrix.shape[1]:
        matrix[x, y] = cuda.threadIdx.x * 100 + cuda.threadIdx.y

In [7]:
matrix = np.zeros((8, 8))
matrix_op[(2, 2), (4, 4)](matrix)
print(matrix)

[[  0.   1.   2.   3.   0.   1.   2.   3.]
 [100. 101. 102. 103. 100. 101. 102. 103.]
 [200. 201. 202. 203. 200. 201. 202. 203.]
 [300. 301. 302. 303. 300. 301. 302. 303.]
 [  0.   1.   2.   3.   0.   1.   2.   3.]
 [100. 101. 102. 103. 100. 101. 102. 103.]
 [200. 201. 202. 203. 200. 201. 202. 203.]
 [300. 301. 302. 303. 300. 301. 302. 303.]]


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:937: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


### Основные функции

- `cuda.grid(1)` - получить глобальный индекс потока (1D)
- `cuda.grid(2)` - получить глобальные индексы (x, y) для 2D
- `cuda.threadIdx.x`, `cuda.threadIdx.y` - индекс потока в блоке
- `cuda.blockIdx.x`, `cuda.blockIdx.y` - индекс блока
- `cuda.blockDim.x`, `cuda.blockDim.y` - размер блока
- `cuda.gridDim.x`, `cuda.gridDim.y` - размер сетки

## Работа с памятью
Еще одно фундаментальное отличие от кода, написанного для выполнения на процессоре -- работа с памятью: нам нужно самим выделять дополнительную память для промежуточных вычислений при работе с GPU, ровно как и загружать данные из памяти компьютера в память GPU.


### Shared memory
Небольшой блок памяти(64-256кБ) в котором мы можем хранить промежуточные вычисления, которыми могут обмениваться между собой потоки.

In [8]:
@cuda.jit
def sum_reduce(arr, out):
    # Создаем shared memory размером 256 элементов
    shared = cuda.shared.array(256, dtype=np.float32)
    
    # Получаем индексы текущего потока
    idx = cuda.grid(1)
    tid = cuda.threadIdx.x
    
    # Копируем данные из глобальной памяти в shared memory
    # Пример: если arr = [1,2,3,4], то shared = [1,2,3,4,0,0,...]
    if idx < arr.size:
        shared[tid] = arr[idx]
    else:
        shared[tid] = 0
    
    # Синхронизируем все потоки
    # Работает так: останавливает данный поток, и ждёт, пока остальные потоки не вызовут эту функцию
    cuda.syncthreads()
    
    # Выполняем параллельную редукцию
    # Пример для массива [1,2,3,4]:
    # Шаг 1: [3,2,7,4,0,0,...] (1+2, 3+4)
    # Шаг 2: [10,2,7,4,0,0,...] (3+7)
    s = 1
    while s < cuda.blockDim.x:
        if tid % (2 * s) == 0:
            shared[tid] += shared[tid + s]
        s *= 2
        cuda.syncthreads()
    
    # Первый поток записывает финальный результат
    # результат = 10 (сумма всех элементов)
    if tid == 0: # смотрим, что это первый элемент
        # out[0] = out[0] + shared[0] - выдаёт неправильный результат из-за параллельного выполнения нескольких потоков
        cuda.atomic.add(out, 0, shared[0])
        
        
n = 1000000
arr = cuda.to_device(np.ones(n, dtype=np.float32))
out = cuda.to_device(np.zeros(1, dtype=np.float32))


threads = 256
blocks = (n + threads - 1) // threads
sum_reduce[blocks, threads](arr, out)

result = out.copy_to_host()
print(f"Sum: {result[0]}")
print(f"Expected: {n}")


Sum: 1000000.0
Expected: 1000000


### Копирование данных CPU -- GPU

In [9]:
@cuda.jit
def multiply_by_two(arr):
    idx = cuda.grid(1)
    if idx < arr.size:
        arr[idx] *= 2


x_host = np.ones(1000) # массив в RAM
x_device = cuda.to_device(x_host) # перенесли на GPU

nulls_device = cuda.device_array(1000) # пустой массив в GPU


threadsperblock = 256
blockspergrid = (x_device.size + (threadsperblock - 1)) // threadsperblock


multiply_by_two[blockspergrid, threadsperblock](x_device)

result = x_device.copy_to_host() # переносим обратно в RAM

print(result[:5])

[2. 2. 2. 2. 2.]


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Стоит понимать, что функции работают именно с загруженными массивами(numba.cuda.cudadrv.devicearray.DeviceNDArray), и если мы вызываем функцию от обычного np.NDArray:
```python
arr = np.zeros(5)
func[1, 5](arr)
```
то numba просто рассахаривает код в что-то типа:
```python
arr = np.zeros(5)
gpu_zeros = cuda.to_device(arr)
func[1, 5](gpu_zeros)
```

## Потоки
Мы можем ещё сильнее углубиться в многопоточность!
Фактически, каждый cuda.steam реализует отдельный поток вычислений. Посмотрим на примерах:

In [10]:
@cuda.jit
def kernel(x, y):
    idx = cuda.grid(1)
    if idx < x.size:
        y[idx] = x[idx] * 2

# Создаём два потока для асинхронного выполнения
stream1 = cuda.stream()
stream2 = cuda.stream()

n = 1000000
x1 = cuda.to_device(np.ones(n))
y1 = cuda.device_array(n)
x2 = cuda.to_device(np.ones(n) * 2)
y2 = cuda.device_array(n)

# Запускаем в разных потоках параллельно
threads = 256
blocks = (n + threads - 1) // threads
kernel[blocks, threads, stream1](x1, y1)
kernel[blocks, threads, stream2](x2, y2)

# Синхронизируем оба потока
stream1.synchronize()  # Ждём завершения stream1
stream2.synchronize()  # Ждём завершения stream2

# Копируем результаты обратно
result1 = y1.copy_to_host()
result2 = y2.copy_to_host()

print("Stream1 result:", result1[:5])
print("Stream2 result:", result2[:5])


Stream1 result: [2. 2. 2. 2. 2.]
Stream2 result: [4. 4. 4. 4. 4.]


Стоит выделить один важный случай использования потоков:
Если есть огромный массив, то можно разбить его на несколько подмассивов поменьше, каждый из которых мы запустим в отдельном stream. 
Это позволяет уменьшить задержки на копирование всего массива в память устройства, позволяя производить вычисления, пока какие-то данные еще передаются.

### Сравнение CPU vs GPU

Простые операции: сложение векторов

In [11]:
@cuda.jit
def add_gpu(x, y, out):
    idx = cuda.grid(1)
    if idx < x.size:
        out[idx] = x[idx] + y[idx]

n = 10000000
x = np.random.rand(n)
y = np.random.rand(n)

print('CPU (NumPy):')
%timeit x + y

x_d = cuda.to_device(x)
y_d = cuda.to_device(y)
out_d = cuda.device_array(n)
threads = 256
blocks = (n + threads - 1) // threads

print('GPU (CUDA):')
%timeit add_gpu[blocks, threads](x_d, y_d, out_d); cuda.synchronize()

CPU (NumPy):
28.2 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
GPU (CUDA):
1.02 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Нахождение суммы элементов:

In [ ]:
n = 100000000
arr = np.random.rand(n)

print('CPU (NumPy sum):')
%timeit np.sum(arr)

arr_d = cuda.to_device(arr.astype(np.float32))
out_d = cuda.to_device(np.zeros(1, dtype=np.float32))
threads = 256
blocks = (n + threads - 1) // threads

print('GPU:')
%timeit sum_reduce[blocks, threads](arr_d, out_d); cuda.synchronize()

CPU (NumPy sum):
67.9 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
GPU:
9.69 ms ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Поэлементные операции:

In [13]:
@cuda.jit
def sin_gpu(x, out):
    idx = cuda.grid(1)
    if idx < x.size:
        out[idx] = cuda.libdevice.sin(x[idx])

n = 10000000
x = np.random.rand(n)

print('CPU (NumPy sin):')
%timeit np.sin(x)

x_d = cuda.to_device(x)
out_d = cuda.device_array(n)
threads = 256
blocks = (n + threads - 1) // threads

print('GPU (CUDA sin):')
%timeit sin_gpu[blocks, threads](x_d, out_d); cuda.synchronize()

CPU (NumPy sin):
134 ms ± 23.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
GPU (CUDA sin):
1.64 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Умножение матриц

In [14]:
@cuda.jit
def matmul_gpu(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

n = 4096
A = np.random.rand(n, n).astype(np.float32)
B = np.random.rand(n, n).astype(np.float32)

print('CPU (NumPy):')
%timeit np.dot(A, B)

A_d = cuda.to_device(A)
B_d = cuda.to_device(B)
C_d = cuda.device_array((n, n), dtype=np.float32)



print('GPU (CUDA):')
%timeit matmul_gpu[(16, 16), (32, 32)](A_d, B_d, C_d); cuda.synchronize()

CPU (NumPy):
1.22 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
GPU (CUDA):
52.9 ms ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
